# Prompt Engineering

Prompt Engineering Techniques explored:

1. System message experimentation
1. Chain-of-Thought with zero-shot and few-shot prompting
1. Condense message experimentation
1. Combining all together (final prompt)
1. Also: Temperature

In [1]:
# Import libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain import OpenAI, ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

from tree_of_thoughts.openaiModels import OpenAILanguageModel
from tree_of_thoughts.treeofthoughts import MonteCarloTreeofThoughts

import tabula

import os

from PyPDF2 import PdfReader
import pandas as df

/home/jethro/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load in openai API key
os.environ['OPENAI_API_KEY'] =""

## Document used and test cases

For the purpose of experimentation, let's use the Tesla Annual Report for year 2022. This document contains both text and tabular information (at the end of document), hence it will be a good test document for our prompt engineering. The document is stored in the "./data" folder.

We should also include some basic test cases beforehand. This will make it easy to check if the prompts are giving us the correct output. Some test cases are defined here for reference.

* Numbers. To test tabular data. 

1. What are the total assets and liabilities for the year 2022? Answer: 82338 million and 36440 million.
1. What is the Income before income taxes for the year  2020, 2021 and 2022? Answer: 1,154, 6,343 and 13,719 respectively.
1. What is the Comprehensive income attributable to common stockholders for year 2022?" Answer: 12141 million
1. What is the gross profit in 2019?
Note: This tests if the model will give a truthful answer or hallucinates the numbers.

* General Questions (open-ended)
1. What does Tesla do? And what types of businesses is Tesla involved in?
1. How is the financial health of Tesla?
1. What kind of technologies does Tesla invest in?

In [77]:
# template code to be used for testing of prompts
query_1 = "What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?"
query_2 = "What is the Income before income taxes for the year  2020, 2021 and 2022?"
query_3 = "What is the Comprehensive income attributable to common stockholders for year 2022?"
query_4 = "What is the gross profit in 2019?"
queries_num = [query_1,query_2,query_3,query_4]

query_5 = "What does Tesla do? And what types of businesses is Tesla involved in?"
query_6 = "How is the financial health of Tesla?"
query_7 = "What kind of technologies does Tesla invest in?"
queries_gen = [query_5, query_6, query_7]

## General Guidelines for prompt engineering

see: https://www.promptingguide.ai/introduction/tips

* Start Simple  
Slowly add on more complexity

* Focus on the instruction  
using commands to instruct the model what you want to achieve, such as "Write", "Classify", "Summarize", "Translate", "Order", etc.  
Another recommendation is to use some clear separator like "###" to separate the instruction and context.


        ### Instruction ###  
        Translate the text below to Spanish:
        Text: "hello!"

        
* Specificity
Be very specific about the instruction and task you want the model to perform. Thinking about how specific and detailed you should be. Including too many unnecessary details is not necessarily a good approach. The details should be relevant and contribute to the task at hand.

* Avoid Impreciseness  
It's often better to be specific and direct. The analogy here is very similar to effective communication -- the more direct, the more effective the message gets across.

* To do or not to do  
Avoid saying what not to do but say what to do instead. This encourages more specificity and focuses on the details that lead to good responses from the model.



## Instruction Prompt Experimentation

Here, we shall experiment with the different kinds of system prompts. To follow the guidelines above. The instruction prompt also forms the system prompt in the ChatPromptTemplate format.



In [4]:
# try baseline case with default prompt template from RetrievalQA
FILE_PATH = "../data/Tesla_Annual_Report_2022.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(data, embeddings)
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())

2023-07-05 19:48:58,648 - INFO - Loading faiss with AVX2 support.
2023-07-05 19:48:58,691 - INFO - Successfully loaded faiss with AVX2 support.


In [5]:
# Inspect what the default prompt message is.
qa.combine_documents_chain.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]

In [6]:
# output using the default prompt message
question = query_7
qa({"query": question})

{'query': 'What kind of technologies does Tesla invest in?',
 'result': 'Tesla invests in a variety of technologies related to automotive, battery and powertrain, vehicle control and infotainment software, self-driving development and artificial intelligence, energy generation and storage, design and engineering, and financial services. Some specific areas of focus include electric powertrain systems, battery cell technology, vehicle control software, self-driving technologies, solar energy systems, and energy storage products. Tesla also places a strong emphasis on intellectual property protection and invests in innovative designs and proprietary technologies.'}

In [7]:
system_template = """### Instruction ###
Use the following pieces of context to answer the users question.
### Guidelines ###
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Your role is as a financial analyst
No matter what the question is, you should always answer it in the context provided below.
If you are unsure of the answer, just say "I do not know"
### context ###
{context}"""

In [8]:
print(system_template)

### Instruction ###
Use the following pieces of context to answer the users question.
### Guidelines ###
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Your role is as a financial analyst
No matter what the question is, you should always answer it in the context provided below.
If you are unsure of the answer, just say "I do not know"
### context ###
{context}


In [9]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_type_kwargs = {"prompt": chat_prompt}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs= chain_type_kwargs)

In [10]:
# output using the revised_1 prompt message
question = query_7
qa({"query": question})

{'query': 'What kind of technologies does Tesla invest in?',
 'result': 'Tesla invests in a variety of technologies related to automotive, battery and powertrain, vehicle control and infotainment software, self-driving development and artificial intelligence, energy generation and storage, design and engineering, and environmental, social, and governance (ESG) initiatives. Some specific areas of investment include powertrain engineering, electric powertrain systems, battery cell chemistry and manufacturing processes, vehicle control software, self-driving technologies, energy storage systems, solar energy systems, intellectual property protection, sustainable manufacturing practices, and customer service and warranty programs.'}

**Observations:**
* As can be seen, the new prompt gives a more detailed answer than the old prompt.
* After checking, it can be seen that both prompts are factually accurate.
* The new prompts, with more instructions and greater precision and clearly defined ## instruction ##, ## guidelines ## and ## context ## does help to give more detailed prompts from the example shown.


## Chain-of-thought with zero-shot prompting and few-shot prompting

What is chain-of-thought prompting: Introduced in Wei et al., chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

Link: https://arxiv.org/abs/2201.11903

In the initial query with base prompt, it was shown that the model had difficulty with query_3 = "What is the year on year increase in the gross profit from 2020 to 2022 in percentages?" As this requires look ahead and higher order thinking. In this example, we try to see if we can use chain-of-thought to resolve this.

In [70]:
# try baseline case with default prompt template from RetrievalQA
def get_text_from_pdf(fs_pdf_docs: list):
    text_output = ""
    for pdf_file in fs_pdf_docs:
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text_output = text_output + page.extract_text()
    return text_output

def get_chunk_from_text(whole_text: str):
    text_split = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", " ", ""],
        chunk_size = 1000,
        chunk_overlap = 200,
        length_function = len
    )
    chunks = text_split.split_text(whole_text)
    return chunks

def get_vectorstore_from_chucks(chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts = chunks, embedding = embeddings)
    return vectorstore

text = get_text_from_pdf([FILE_PATH])
chunks = get_chunk_from_text(text)
vector_store = get_vectorstore_from_chucks(chunks)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())

In [12]:
# output using the default prompt message
# As can be seen, the model got lost in the context and is unable to even retrieve information on the gross profit
question = query_3
qa({"query": question})

{'query': 'What is the year on year increase in the gross profit from 2020 to 2022 in percentages?',
 'result': 'The year-on-year increase in gross profit from 2020 to 2022 is not provided in the given context.'}

In [13]:
# Try with COT with zero-shot prompting by adding "let;s think this step by step"
system_template = "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer. Let's think step by step.\n----------------\n{context}"

In [14]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_type_kwargs = {"prompt": chat_prompt}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs= chain_type_kwargs)

In [15]:
# There is an improvement in the output. However, it response is still not complete. 
question = query_3
qa({"query": question})

{'query': 'What is the year on year increase in the gross profit from 2020 to 2022 in percentages?',
 'result': 'To calculate the year-on-year increase in gross profit from 2020 to 2022 in percentages, we need the gross profit figures for both years. However, the provided information only includes the gross profit figures for 2021 and 2022. Without the gross profit figure for 2020, we cannot calculate the exact year-on-year increase in percentages.'}

**Observations:**
* The base prompt gave the worst reply. It did not manage to get the figures of gross profit from year 2020. There could be an issue with the way the data is read in or the similarity search with the context.
* The zero-shot CoT prompt gave a much improved reply, showing that it understood the context and the way to calculate the figures. However, the figures are not correct. Upon further inspection, there it read the data from the wrong table.

In [16]:
# Let's introduce CoT with few-shot prompting
system_template = """Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Let's think step by step.
---------------------
Q: What is the percentage increase in gross profit margin from year 2021 to year 2022.
A: The gross profit margin in year 2021 is 13606. The gross profit margin in year 2022 is 20853. Using the formula Percentage Increase = ((New Value - Old Value) / Old Value), the gross profit margin is 53.26%.
Q: What is the net income from financial year 2021 and 2022 combined?
A: The net income for financial year 2021 is 300 million. The net income for financial year 2022 is 200 million. Therefore, the combined net income is 500 million.
### context ###
{context}"""

print(system_template)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_type_kwargs = {"prompt": chat_prompt}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.0), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs= chain_type_kwargs)

Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Let's think step by step.
---------------------
Q: What is the percentage increase in gross profit margin from year 2021 to year 2022.
A: The gross profit margin in year 2021 is 13606. The gross profit margin in year 2022 is 20853. Using the formula Percentage Increase = ((New Value - Old Value) / Old Value), the gross profit margin is 53.26%.
Q: What is the net income from financial year 2021 and 2022 combined?
A: The net income for financial year 2021 is 300 million. The net income for financial year 2022 is 200 million. Therefore, the combined net income is 500 million.
### context ###
{context}


In [17]:
# Try another question for CoT with few-shot prompting
question = " What is the gross profit for year 2021 and year 2022 combined?"
qa({"query": question})

{'query': ' What is the gross profit for year 2021 and year 2022 combined?',
 'result': 'The gross profit for the year 2021 is $13,735 million. The gross profit for the year 2022 is $20,565 million. Therefore, the combined gross profit for the two years is $34,300 million.'}

In [18]:
# Try with base prompt
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())
question = " What is the gross profit for year 2021 and year 2022 combined?"
qa({"query": question})

{'query': ' What is the gross profit for year 2021 and year 2022 combined?',
 'result': 'The gross profit for the years 2021 and 2022 combined is $27,470 million.'}

In [19]:
qa.combine_documents_chain.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]

**Observations:**
* The CoT with few-shot prompt is able to explain out the reasoning of how it got to the final answer. However, it read the data from the wrong table. (See page 88 on segment reporting)
* The base prompt did not explain the thought process and gave a wrong answer as well.

## Condense QA Prompt
The ConversationalRetrievalChain is a component of the Langchain system that combines chat history, question condensing, semantic search, and question answering to provide a response. On the backend, the ConversationalRetrievalChain performs the following steps:

It condenses the current question and the chat history into a standalone question. This is done to create a standalone vector for retrieval.

It uses a retriever, which can be created from a vector store, to look up relevant documents based on the condensed question.

It passes the retrieved documents and the original question to a question answering chain to generate a response.

It returns the answer to the user.

* We can update the condense_QA_prompt that performs step 1 to summarize the current question and the chat history.

In [20]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    output_key="answer",
    return_generated_question = True
)

In [21]:
# Inspect the default QA prompt
chain.question_generator.prompt

PromptTemplate(input_variables=['chat_history', 'question'], output_parser=None, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:', template_format='f-string', validate_template=True)

In [30]:
# Let's use the base prompt to see how the output standalone question looks like
chat_history = []
question = query_7
print(question)
result = chain({"question": question, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_1 = "Out of all the technologies, which one is the most interesting and has the most potential?"
print(sub_question_1)
result = chain({"question": sub_question_1, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_2 = "Tell me more about the autonomous driving technology."
print(sub_question_2)
result = chain({"question": sub_question_2, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))


What kind of technologies does Tesla invest in?
Tesla invests in a variety of technologies, including electric vehicle technology, autonomous driving technology, energy generation and storage technology, and artificial intelligence. They continuously develop and improve their electric vehicle technology, including advanced driver assist systems and autonomous driving capabilities. They also invest in energy storage products, such as Powerwall and Megapack, which utilize lithium-ion battery technology. Additionally, Tesla applies their artificial intelligence learnings to robotics and has showcased a robotic humanoid controlled by their AI system.
-------------------------------------------------
Out of all the technologies, which one is the most interesting and has the most potential?
Based on the given context, it is difficult to determine which technology has the most potential as it depends on individual perspectives and priorities. However, some technologies mentioned that have sig

In [31]:
# the output after condensing history and new question
result['generated_question']

"What are some details about Tesla's autonomous driving technology?"

In [32]:
memory_template = """
Given the following chat history and a follow up question, rephrase the\
follow up question to be a standalone question.\
The follow up question may not always be based on the chat history.\
If follow up question is not based on the chat history, do not rephrase it.\
If follow up question is not based on the chat history, you should still answer it\
in the context given below.\
If the question is not related to the context below, just say that "I don't know".\
Chat History:{chat_history}\
Follow Up Question: {question}\
Standalone Question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(memory_template)

In [33]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    output_key="answer",
    return_generated_question = True
)

In [34]:
# Let's use the improved prompt to see how the output standalone question looks like
chat_history = []
question = query_7
print(question)
result = chain({"question": question, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_1 = "Out of all the technologies, which one is the most interesting and has the most potential?"
print(sub_question_1)
result = chain({"question": sub_question_1, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_2 = "Tell me more about the autonomous driving technology."
print(sub_question_2)
result = chain({"question": sub_question_2, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))

What kind of technologies does Tesla invest in?
Tesla invests in a variety of technologies related to electric vehicles, energy generation, and storage. Some of the key technologies they focus on include:

1. Electric Vehicle Technology: Tesla invests heavily in developing advanced electric vehicle technology, including battery technology, electric drivetrains, and autonomous driving systems.

2. Battery Technology: Tesla is known for its expertise in battery technology and invests in developing high-performance and long-lasting lithium-ion batteries for its vehicles and energy storage products.

3. Autonomous Driving: Tesla is at the forefront of autonomous driving technology and invests in developing advanced driver-assist systems and full self-driving capabilities for its vehicles.

4. Energy Storage: Tesla invests in energy storage technology, including the development of large-scale battery systems like the Megapack, which can store and distribute renewable energy.

5. Solar Energ

In [35]:
# the output after condensing history and new question
# for improved condensed qa prompt
result['generated_question']

"What are some key features and advancements in Tesla's autonomous driving technology?"

## Combining all together (final prompt template used)
Now that we have experimented with different methods of prompt engineering and observed the outputs, we can combine the knowledge to generate the eventual prompt used for the app deployment.

**Main observations from prompt engineering:**
1. Modifying the system prompt with instructions, guidelines and context gives answers with more details and nuances. The answers are factual and accurate.
1. CoT with zero-shot prompting is able to produce answers which are relevant and contextual. It is also highly verbose and explains with rich details on how it arrived at the final answer.
1. CoT with few-shot prompting gave similar answers to CoT with zero-shot prompting. This method also used more tokens, leaving less token space available for the context. Hence, it is better not to use CoT with few-shot prompting for our use case.

In [81]:
# QA prompt
system_template = """### Instruction ###
Use the following pieces of context to answer the users question.
### Guidelines ###
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Your role is as a financial analyst
No matter what the question is, you should always answer it in the context provided below.
If you are unsure of the answer, just say "I do not know"
### context ###
{context}"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain_type_kwargs = {"prompt": chat_prompt}


In [82]:
# Condensed QA prompt
memory_template = """
Given the following chat history and a follow up question, rephrase the\
follow up question to be a standalone question.\
The follow up question may not always be based on the chat history.\
If follow up question is not based on the chat history, do not rephrase it.\
If follow up question is not based on the chat history, you should still answer it\
in the context given below.\
If the question is not related to the context below, just say that "I don't know".\
Chat History:{chat_history}\
Follow Up Question: {question}\
Standalone Question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(memory_template)

In [85]:
# Final Chain
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    output_key="answer",
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs= chain_type_kwargs
)


In [91]:
result = chain({"question": question, "chat_history": chat_history})

chat_history.append((question, result["answer"]))

result["source_documents"]

[Document(page_content='We also work with a wide variety of hospitality, retail and public destinations, as well as businesses with commuting employees,\nto offer additional charging options for our customers, as well as single-family homeowners and multi-family residential entities, todeploy home charging solutions.7In-App Upgrades\nAs our vehicles are capable of being updated remotely over-the-air, our customers may purchase additional paid options and\nfeatures through the Tesla app or through the in-vehicle user interface. We expect that this functionality will also allow us to offercertain options and features on a subscription basis in the future.\nEnergy Generation and Storage\nWe market and sell our solar and energy storage products to residential, commercial and industrial customers and utilities', metadata={}),
 Document(page_content='Energy Generation and Storage\nWe market and sell our solar and energy storage products to residential, commercial and industrial customers and

In [92]:
type(result["source_documents"])

list

Now that we have created our final prompt template, let's run through all the 7 queries and see the output of them.

In [87]:
for query in queries_num:
    chat_history = []
    result = chain({"question": query, "chat_history": chat_history})
    print(result['question'])
    print(result['answer'])
    print("---------------------------------------------")

What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the consolidated balance sheet are $82,338 million. The total liabilities for the year 2022 in the consolidated balance sheet are $36,440 million.
---------------------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the year 2020 was $1,154 million, for the year 2021 was $6,343 million, and for the year 2022 was $13,719 million.
---------------------------------------------
What is the Comprehensive income attributable to common stockholders for year 2022?
The Comprehensive income attributable to common stockholders for the year 2022 is $12,141 million.
---------------------------------------------
What is the gross profit in 2019?
I do not know the gross profit in 2019 as the provided context only includes information for the years 2020, 2021, and 2022.
----------------

In [88]:
for query in queries_gen:
    chat_history = []
    result = chain({"question": query, "chat_history": chat_history})
    print(result['question'])
    print(result['answer'])
    print("---------------------------------------------")

What does Tesla do? And what types of businesses is Tesla involved in?
Tesla is a company that primarily focuses on the design, development, manufacturing, and sale of electric vehicles. They also provide energy generation and storage solutions through their solar and energy storage products. Additionally, Tesla offers services such as vehicle maintenance and repairs.

In terms of the types of businesses Tesla is involved in, they work with a wide range of industries. They collaborate with hospitality, retail, and public destinations to provide charging options for their customers. They also work with businesses that have commuting employees to offer charging solutions. Furthermore, Tesla markets and sells their solar and energy storage products to residential, commercial, and industrial customers, as well as utilities.
---------------------------------------------
How is the financial health of Tesla?
As a financial analyst, I can provide some insights into Tesla's financial health ba